In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Vezilka/SpisokOpstiniMK.csv", encoding="utf-8", skiprows=3)
df.head()

,Матичен број на општина,Назив на општина,Матичен број на населено место,Назив на населено место,Град (г) / Село (с),Unnamed: 5
0,0019,Град Скопје*,499994.0,Скопје,г,NaN
1,0094,Аеродром,419478.0,Долно Лисиче,с,NaN
2,0094,Аеродром,491098.0,Скопје - Аеродром,г,NaN
3,0108,Бутел,419516.0,Визбегово,с,NaN
4,0108,Бутел,414999.0,Љубанци,с,NaN


In [ ]:
df = df.iloc[:, :-1]

In [ ]:
df.head()

,Матичен број на општина,Назив на општина,Матичен број на населено место,Назив на населено место,Град (г) / Село (с)
0,0019,Град Скопје*,499994.0,Скопје,г
1,0094,Аеродром,419478.0,Долно Лисиче,с
2,0094,Аеродром,491098.0,Скопје - Аеродром,г
3,0108,Бутел,419516.0,Визбегово,с
4,0108,Бутел,414999.0,Љубанци,с


In [ ]:
df.columns = [
    "opstina_id",
    "opstina",
    "sifra_nas_mesto",
    "naseleno_mesto",
    "tip_naseleno_mesto"
]

In [ ]:
print(df.columns)
print(df.shape)
df.isna().sum()


Index(['opstina_id', 'opstina', 'sifra_nas_mesto', 'naseleno_mesto',
       'tip_naseleno_mesto'],
      dtype='object')
(1779, 5)


,0
opstina_id,1
opstina,2
sifra_nas_mesto,2
naseleno_mesto,2
tip_naseleno_mesto,2


In [ ]:
df = df.dropna(subset=["naseleno_mesto", "opstina", "tip_naseleno_mesto"])
df.shape

(1777, 5)

In [ ]:
df["tip_naseleno_mesto"] = (
    df["tip_naseleno_mesto"]
    .astype(str)
    .str.strip()
    .replace({"г": "град", "с": "село"})
)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1777 entries, 0 to 1776
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   opstina_id          1777 non-null   object 
 1   opstina             1777 non-null   object 
 2   sifra_nas_mesto     1777 non-null   float64
 3   naseleno_mesto      1777 non-null   object 
 4   tip_naseleno_mesto  1777 non-null   object 
dtypes: float64(1), object(4)
memory usage: 83.3+ KB


In [ ]:
!pip -q install "argilla>=2.0.0" pandas tqdm huggingface_hub openai


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.3/161.3 kB 4.8 MB/s eta 0:00:00


In [ ]:
import argilla as rg
from google.colab import userdata

ARGILLA_API_URL = (userdata.get("ARGILLA_API_URL") or "").strip()
ARGILLA_API_KEY = (userdata.get("ARGILLA_API_KEY") or "").strip()

print("URL repr:", repr(ARGILLA_API_URL))
print("KEY starts with:", ARGILLA_API_KEY[:10])

client = rg.Argilla(api_url=ARGILLA_API_URL, api_key=ARGILLA_API_KEY)
print(client.me)

URL repr: 'https://miloshevskamaja-mk-argilla.hf.space'
KEY starts with: N9L-QfEEmC
User(id=UUID('7d00a144-0cef-4a4c-80b4-6d1823b7fbca') inserted_at=datetime.datetime(2026, 1, 30, 19, 53, 44, 568435) updated_at=datetime.datetime(2026, 1, 30, 19, 53, 44, 568435) username='miloshevskamaja' role=<Role.owner: 'owner'> first_name='miloshevskamaja' last_name=None password=None)


In [ ]:
required_cols = ["opstina_id", "opstina",  "sifra_nas_mesto", "naseleno_mesto","tip_naseleno_mesto"]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns in df: {missing}")

df2 = df.copy()

df2["sifra_nas_mesto"] = df2["sifra_nas_mesto"].astype("Int64").astype(str)

for c in ["naseleno_mesto","opstina"]:
    df2[c] = df2[c].astype(str).str.strip()

df2 = df2.dropna(subset=["naseleno_mesto","opstina","tip_naseleno_mesto"]).reset_index(drop=True)
df2.shape, df2.head(3)

((1777, 5),
   opstina_id       opstina sifra_nas_mesto     naseleno_mesto  \
 0       0019  Град Скопје*          499994             Скопје   
 1       0094      Аеродром          419478       Долно Лисиче   
 2       0094      Аеродром          491098  Скопје - Аеродром   
 
   tip_naseleno_mesto  
 0               град  
 1               село  
 2               град  )

Part 1 - with LLM MistralAi - bad data with hallucination and confusion between Macedonian, Bulgarian and Serbian language  

In [ ]:
def make_prompt(row, n_sentences=2):
    return (
        f"Генерирај {n_sentences} кратки, различни реченици на македонски јазик, "
        f"строго базирани на следниве факти (не додавај ништо друго):\n"
        f"- Населено место: {row['naseleno_mesto']}\n"
        f"- Тип: {row['tip_naseleno_mesto']}\n"
        f"- Општина: {row['opstina']}\n"
        "Правила:\n"
        "1) Не измислувај популација, координати, региони, историја.\n"
        "2) Не користи англиски.\n"
        "3) Врати само речениците (секојa во нов ред).\n"
        "4) Ако некој податок недостига, немој да претпоставуваш – користи само даденото.\n"
    )

In [ ]:
from huggingface_hub import InferenceClient

HF_TOKEN = userdata.get("HF_TOKEN")

client_hf = InferenceClient(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    token=HF_TOKEN
)

In [ ]:
def gen_hf_chat(prompt, temperature=0.6, max_tokens=220):
    resp = client_hf.chat.completions.create(
        messages=[
            {"role": "system", "content": "Пишувај само на македонски (кирилица). Биди строго фактолошки."},
            {"role": "user", "content": prompt},
        ],
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return resp.choices[0].message.content.strip()


In [ ]:
from tqdm import tqdm

N = 200
sample_df = df2.sample(min(N, len(df2)), random_state=42).reset_index(drop=True)

prompts, generations = [], []

for _, row in tqdm(sample_df.iterrows(), total=len(sample_df), desc="Generating (HF chat)"):
    prompt = make_prompt(row, n_sentences=2)
    prompts.append(prompt)

    try:
        text = gen_hf_chat(prompt)
    except Exception as e:
        text = f"[ERROR] {type(e).__name__}: {e}"

    generations.append(text)

sample_df["prompt"] = prompts
sample_df["generation"] = generations


Generating (HF chat): 100%|██████████| 200/200 [21:36<00:00,  6.48s/it]


In [ ]:
generations

['1. В Орланциот живи и селувјат хора наречени Арачиновци.\n2. Селото Арачиново е част от община Орланци.',
 '1. Село Липково е част од општина Стрима.\n2. Липково - село в Стримска регија, Северна Македонија.',
 '1. Село Крива Паланка е част от община Нерав.\n2. Крива Паланка е селце в Община Нерав.',
 '1. Село Чашка се налази во општина Раковец.\n2. Раковец е општина где живут хора во село Чашка.',
 '1. Село Кратово е част от општина Татомир.\n2. Кратово - село в границите на община Татомир.',
 '1. Село Струга е част от община Дренок.\n2. Дренокската община е дома на село Струга.',
 '1. Македонски Брод е град, седетел на Община Македонски Брод.\n2. Градският център Македонски Брод е част от Община със същото име.',
 '1. Село в општина Горно Оризари, Македонија - това е Велес.\n2. Велешко село се налази во Горно Оризари.',
 '1. Село Петровец е част от община Дивље.\n2. Петровец е село в община Дивље.',
 '1. Село Старо Нагоричане се налази в општина Арбанашко.\n2. У централната част на

Part 2 - Template Based Generation

In [ ]:
import random

TEMPLATES = [
    lambda r: [
        f"„{r['naseleno_mesto']}“ е {r['tip_naseleno_mesto']} во општина „{r['opstina']}“.",
        f"Населеното место „{r['naseleno_mesto']}“ припаѓа на општина „{r['opstina']}“."
    ],
    lambda r: [
        f"{r['tip_naseleno_mesto'].capitalize()}то „{r['naseleno_mesto']}“ се наоѓа во општина „{r['opstina']}“.",
        f"Општина „{r['opstina']}“ го опфаќа населеното место „{r['naseleno_mesto']}“."
    ],
    lambda r: [
        f"„{r['naseleno_mesto']}“ е населено место од тип {r['tip_naseleno_mesto']} во општина „{r['opstina']}“.",
        f"Како {r['tip_naseleno_mesto']}, „{r['naseleno_mesto']}“ административно припаѓа на општина „{r['opstina']}“."
    ],
    lambda r: [
        f"Во општина „{r['opstina']}“ се наоѓа {r['tip_naseleno_mesto']}то „{r['naseleno_mesto']}“.",
        f"„{r['naseleno_mesto']}“ е {r['tip_naseleno_mesto']} кое е дел од општина „{r['opstina']}“."
    ],
]




In [ ]:
def gen_template_var(row, seed=None):
    if seed is not None:
        random.seed(seed)
    tmpl = random.choice(TEMPLATES)
    lines = tmpl(row)
    return "\n".join(lines)

In [ ]:
from tqdm import tqdm
import pandas as pd

N = len(df2)
sample_df = df2.sample(N, random_state=42).reset_index(drop=True)

generations = []

for _, row in tqdm(sample_df.iterrows(), total=len(sample_df), desc="Template-based generation"):
    text = gen_template_var(row)
    generations.append(text)

sample_df["generation"] = generations

sample_df[["naseleno_mesto", "opstina", "tip_naseleno_mesto", "generation"]].head(10)


Template-based generation: 100%|██████████| 1777/1777 [00:00<00:00, 22544.36it/s]


,naseleno_mesto,opstina,tip_naseleno_mesto,generation
0,Орланци,Арачиново,село,Селото „Орланци“ се наоѓа во општина „Арачинов...
1,Стрима,Липково,село,„Стрима“ е населено место од тип село во општи...
2,Нерав,Крива Паланка,село,Во општина „Крива Паланка“ се наоѓа селото „Не...
3,Раковец,Чашка,село,„Раковец“ е село во општина „Чашка“.\nНаселено...
4,Татомир,Кратово,село,„Татомир“ е населено место од тип село во општ...
5,Дренок,Струга,село,Во општина „Струга“ се наоѓа селото „Дренок“.\...
6,Македонски Брод,Македонски Брод,град,Градто „Македонски Брод“ се наоѓа во општина „...
7,Горно Оризари,Велес,село,„Горно Оризари“ е населено место од тип село в...
8,Дивље,Петровец,село,„Дивље“ е село во општина „Петровец“.\nНаселен...
9,Арбанашко,Старо Нагоричане,село,Селото „Арбанашко“ се наоѓа во општина „Старо ...


In [ ]:
generations


['„Орланци“ е село во општина „Арачиново“.\nНаселеното место „Орланци“ припаѓа на општина „Арачиново“.',
 '„Стрима“ е село во општина „Липково“.\nНаселеното место „Стрима“ припаѓа на општина „Липково“.',
 '„Нерав“ е населено место од тип село во општина „Крива Паланка“.\nКако село, „Нерав“ административно припаѓа на општина „Крива Паланка“.',
 'Во општина „Чашка“ се наоѓа селото „Раковец“.\n„Раковец“ е село кое е дел од општина „Чашка“.',
 '„Татомир“ е населено место од тип село во општина „Кратово“.\nКако село, „Татомир“ административно припаѓа на општина „Кратово“.',
 '„Дренок“ е село во општина „Струга“.\nНаселеното место „Дренок“ припаѓа на општина „Струга“.',
 'Градто „Македонски Брод“ се наоѓа во општина „Македонски Брод“.\nОпштина „Македонски Брод“ го опфаќа населеното место „Македонски Брод“.',
 'Селото „Горно Оризари“ се наоѓа во општина „Велес“.\nОпштина „Велес“ го опфаќа населеното место „Горно Оризари“.',
 '„Дивље“ е село во општина „Петровец“.\nНаселеното место „Дивље“ при

In [ ]:
import json

def make_sft_prompt(row):
    return (
        f"Населено место: {row['naseleno_mesto']}\n"
        f"Тип: {row['tip_naseleno_mesto']}\n"
        f"Општина: {row['opstina']}\n"
        f"Напиши две фактолошки реченици на македонски."
    )

records = []
for _, row in sample_df.iterrows():
    records.append({
        "prompt": make_sft_prompt(row),
        "completion": row["generation"]
    })

with open("mk_settlements_template_sft.jsonl", "w", encoding="utf-8") as f:
    for r in records:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

print("Saved: mk_settlements_template_sft.jsonl")


Saved: mk_settlements_template_sft.jsonl


Part 3 - Generate synthetic data using KGs

In [ ]:
import pandas as pd

In [ ]:
def build_triples(df):
    triples = []
    for _, r in df.iterrows():
        nm = r["naseleno_mesto"]
        op = r["opstina"]
        typ = r["tip_naseleno_mesto"]

        triples.append((nm, "е_тип", typ))
        triples.append((nm, "припаѓа_на", op))
        triples.append((op, "има_населено_место", nm))
    return pd.DataFrame(triples, columns=["subj", "pred", "obj"])

In [ ]:
triples_df = build_triples(df2)
triples_df.head(10), triples_df.shape

(                subj                pred                obj
 0             Скопје               е_тип               град
 1             Скопје          припаѓа_на       Град Скопје*
 2       Град Скопје*  има_населено_место             Скопје
 3       Долно Лисиче               е_тип               село
 4       Долно Лисиче          припаѓа_на           Аеродром
 5           Аеродром  има_населено_место       Долно Лисиче
 6  Скопје - Аеродром               е_тип               град
 7  Скопје - Аеродром          припаѓа_на           Аеродром
 8           Аеродром  има_населено_место  Скопје - Аеродром
 9          Визбегово               е_тип               село,
 (5331, 3))

In [ ]:
from collections import defaultdict

In [ ]:
# општина -> листа на населени места
op_to_nm = defaultdict(list)

# населено место -> општина
nm_to_op = {}

# населено место -> тип
nm_to_type = {}

In [ ]:
for _, r in df2.iterrows():
    nm = r["naseleno_mesto"]
    op = r["opstina"]
    typ = r["tip_naseleno_mesto"]
    op_to_nm[op].append(nm)
    nm_to_op[nm] = op
    nm_to_type[nm] = typ

In [ ]:
import random

def kg_text_for_row(row, seed=None):
    if seed is not None:
        random.seed(seed)

    nm = row["naseleno_mesto"]
    op = row["opstina"]
    typ = row["tip_naseleno_mesto"]

    neighbors = [x for x in op_to_nm[op] if x != nm]
    other = random.choice(neighbors) if neighbors else None
    other_typ = nm_to_type.get(other) if other else None

    candidates = []

    candidates.append(
        "\n".join([
            f"„{nm}“ е {typ} во општина „{op}“.",
            f"Општина „{op}“ го опфаќа населеното место „{nm}“."
        ])
    )

    candidates.append(
        "\n".join([
            f"Во општина „{op}“ се наоѓа {typ}то „{nm}“.",
            f"Населеното место „{nm}“ административно припаѓа на општина „{op}“."
        ])
    )

    if other:
        candidates.append(
            "\n".join([
                f"„{nm}“ е {typ} во општина „{op}“.",
                f"Во истата општина се наоѓа и {other_typ}то „{other}“."
            ])
        )
        candidates.append(
            "\n".join([
                f"Општина „{op}“ ги опфаќа населените места „{nm}“ и „{other}“.",
                f"„{nm}“ е {typ}, а „{other}“ е {other_typ}."
            ])
        )

    if other:
        candidates.append(
            "\n".join([
                f"Во општина „{op}“ се наоѓаат „{nm}“ и „{other}“.",
                f"„{nm}“ е {typ}, додека „{other}“ е {other_typ}."
            ])
        )

    return random.choice(candidates)


In [ ]:
from tqdm import tqdm

kg_generations = []
for _, row in tqdm(sample_df.iterrows(), total=len(sample_df), desc="KG-based generation"):
    kg_generations.append(kg_text_for_row(row))

sample_df["kg_generation"] = kg_generations

sample_df[["naseleno_mesto","opstina","tip_naseleno_mesto","generation","kg_generation"]].head(10)


KG-based generation: 100%|██████████| 1777/1777 [00:00<00:00, 13110.80it/s]


,naseleno_mesto,opstina,tip_naseleno_mesto,generation,kg_generation
0,Орланци,Арачиново,село,Селото „Орланци“ се наоѓа во општина „Арачинов...,„Орланци“ е село во општина „Арачиново“.\nВо и...
1,Стрима,Липково,село,„Стрима“ е населено место од тип село во општи...,Во општина „Липково“ се наоѓа селото „Стрима“....
2,Нерав,Крива Паланка,село,Во општина „Крива Паланка“ се наоѓа селото „Не...,Во општина „Крива Паланка“ се наоѓаат „Нерав“ ...
3,Раковец,Чашка,село,„Раковец“ е село во општина „Чашка“.\nНаселено...,Општина „Чашка“ ги опфаќа населените места „Ра...
4,Татомир,Кратово,село,„Татомир“ е населено место од тип село во општ...,„Татомир“ е село во општина „Кратово“.\nВо ист...
5,Дренок,Струга,село,Во општина „Струга“ се наоѓа селото „Дренок“.\...,Во општина „Струга“ се наоѓа селото „Дренок“.\...
6,Македонски Брод,Македонски Брод,град,Градто „Македонски Брод“ се наоѓа во општина „...,„Македонски Брод“ е град во општина „Македонск...
7,Горно Оризари,Велес,село,„Горно Оризари“ е населено место од тип село в...,„Горно Оризари“ е село во општина „Велес“.\nВо...
8,Дивље,Петровец,село,„Дивље“ е село во општина „Петровец“.\nНаселен...,Во општина „Петровец“ се наоѓа селото „Дивље“....
9,Арбанашко,Старо Нагоричане,село,Селото „Арбанашко“ се наоѓа во општина „Старо ...,Во општина „Старо Нагоричане“ се наоѓа селото ...


In [ ]:
kg_generations

['„Орланци“ е село во општина „Арачиново“.\nВо истата општина се наоѓа и селото „Арачиново“.',
 'Во општина „Липково“ се наоѓа селото „Стрима“.\nНаселеното место „Стрима“ административно припаѓа на општина „Липково“.',
 'Во општина „Крива Паланка“ се наоѓаат „Нерав“ и „Станци“.\n„Нерав“ е село, додека „Станци“ е село.',
 'Општина „Чашка“ ги опфаќа населените места „Раковец“ и „Извор“.\n„Раковец“ е село, а „Извор“ е село.',
 '„Татомир“ е село во општина „Кратово“.\nВо истата општина се наоѓа и градто „Кратово“.',
 'Во општина „Струга“ се наоѓа селото „Дренок“.\nНаселеното место „Дренок“ административно припаѓа на општина „Струга“.',
 '„Македонски Брод“ е град во општина „Македонски Брод“.\nВо истата општина се наоѓа и селото „Долно Ботушје“.',
 '„Горно Оризари“ е село во општина „Велес“.\nВо истата општина се наоѓа и селото „Башино Село“.',
 'Во општина „Петровец“ се наоѓа селото „Дивље“.\nНаселеното место „Дивље“ административно припаѓа на општина „Петровец“.',
 'Во општина „Старо Наго

In [ ]:
rows = []
for _, r in sample_df.iterrows():
    base = {
        "opstina_id": r["opstina_id"],
        "sifra_nas_mesto": r["sifra_nas_mesto"],
        "naseleno_mesto": r["naseleno_mesto"],
        "opstina": r["opstina"],
        "tip_naseleno_mesto": r["tip_naseleno_mesto"],
    }
    rows.append({**base, "source": "template", "text": r["generation"]})
    rows.append({**base, "source": "kg", "text": r["kg_generation"]})

train_df = pd.DataFrame(rows)


In [ ]:
train_df.shape

(3554, 7)

In [ ]:
train_df.head(5)

,opstina_id,sifra_nas_mesto,naseleno_mesto,opstina,tip_naseleno_mesto,source,text
0,1015,414026,Орланци,Арачиново,село,template,Селото „Орланци“ се наоѓа во општина „Арачинов...
1,1015,414026,Орланци,Арачиново,село,kg,„Орланци“ е село во општина „Арачиново“.\nВо и...
2,1694,409430,Стрима,Липково,село,template,„Стрима“ е населено место од тип село во општи...
3,1694,409430,Стрима,Липково,село,kg,Во општина „Липково“ се наоѓа селото „Стрима“....
4,1627,407925,Нерав,Крива Паланка,село,template,Во општина „Крива Паланка“ се наоѓа селото „Не...


In [ ]:
train_df['text'][15]

'„Горно Оризари“ е село во општина „Велес“.\nВо истата општина се наоѓа и селото „Башино Село“.'

In [ ]:
import json

def make_prompt_from_row(r):
    return (
        f"Населено место: {r['naseleno_mesto']}\n"
        f"Тип: {r['tip_naseleno_mesto']}\n"
        f"Општина: {r['opstina']}\n"
        f"Напиши две фактолошки реченици на македонски."
    )

out_path = "synthetic_sententes_kg.jsonl"
with open(out_path, "w", encoding="utf-8") as f:
    for _, r in train_df.iterrows():
        rec = {
            "prompt": make_prompt_from_row(r),
            "completion": r["text"],
            "meta": {"source": r["source"]}
        }
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")

print("Saved:", out_path)


Saved: synthetic_sententes_kg.jsonl
